# Week 2: Data Cleaning & Preprocessing  
### Dataset: Student Performance Insights  
This notebook covers the Week 2 tasks for cleansing and preprocessing the student performance dataset using SQL on Databricks Free Edition (Serverless SQL Warehouse).

## Objectives
- Load The dataset 
- Identify duplicates  
- Handle missing values  
- Standardize categorical fields  
- Create a cleaned final table  
- Prepare the dataset for Week 3 EDA

In [0]:
%sql
USE CATALOG student_performance_insights;
USE SCHEMA default;

SELECT * FROM students_performance LIMIT 10;

gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
female,group B,bachelor's degree,standard,none,72,72,74
female,group C,some college,standard,completed,69,90,88
female,group B,master's degree,standard,none,90,95,93
male,group A,associate's degree,free/reduced,none,47,57,44
male,group C,some college,standard,none,76,78,75
female,group B,associate's degree,standard,none,71,83,78
female,group B,some college,standard,completed,88,95,92
male,group B,some college,free/reduced,none,40,43,39
male,group D,high school,free/reduced,completed,64,64,67
female,group B,high school,free/reduced,none,38,60,50


## Identify & Remove Duplicates
We check for duplicate records using SQL `GROUP BY` and then recreate the table using `SELECT DISTINCT` to remove any duplicates.

In [0]:
%sql
SELECT `gender`, `race/ethnicity`, `parental level of education`, `lunch`, 
        `test preparation course`, `math score`, `reading score`, `writing score`,
       COUNT(*) AS count
FROM student_performance_insights.default.students_performance
GROUP BY `gender`, `race/ethnicity`, `parental level of education`, `lunch`, 
         `test preparation course`, `math score`, `reading score`, `writing score`
HAVING COUNT(*) > 1;

gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,count


## 3. Handle Missing Values
Missing values can occur due to incomplete data entry or import issues.  
We detect NULL values for each column and apply appropriate default replacements using `COALESCE()`.

In [0]:
%sql
SELECT 
  SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS missing_gender,
  SUM(CASE WHEN `race/ethnicity` IS NULL THEN 1 ELSE 0 END) AS missing_race,
  SUM(CASE WHEN `parental level of education` IS NULL THEN 1 ELSE 0 END) AS missing_parent_education,
  SUM(CASE WHEN lunch IS NULL THEN 1 ELSE 0 END) AS missing_lunch,
  SUM(CASE WHEN `test preparation course` IS NULL THEN 1 ELSE 0 END) AS missing_prep,
  SUM(CASE WHEN `math score` IS NULL THEN 1 ELSE 0 END) AS missing_math,
  SUM(CASE WHEN `reading score` IS NULL THEN 1 ELSE 0 END) AS missing_reading,
  SUM(CASE WHEN `writing score` IS NULL THEN 1 ELSE 0 END) AS missing_writing
FROM student_performance_insights.default.students_performance;

missing_gender,missing_race,missing_parent_education,missing_lunch,missing_prep,missing_math,missing_reading,missing_writing
0,0,0,0,0,0,0,0


## 4. Standardize Categorical Columns
Inconsistent text entries can create issues during analysis.  
We use `INITCAP()` to ensure consistent formatting across categorical fields like:
- Gender  
- Race/Ethnicity  
- Parental education  
- Lunch type  
- Test preparation status

In [0]:
%sql
SELECT
  INITCAP(`gender`) AS gender,
  INITCAP(`race/ethnicity`) AS race_ethnicity,
  INITCAP(`parental level of education`) AS parent_education,
  INITCAP(`lunch`) AS lunch,
  INITCAP(`test preparation course`) AS test_prep,
  `math score` AS math_score,
  `reading score` AS reading_score,
  `writing score` AS writing_score
FROM student_performance_insights.default.students_performance
LIMIT 10;

gender,race_ethnicity,parent_education,lunch,test_prep,math_score,reading_score,writing_score
Female,Group B,Bachelor's Degree,Standard,None,72,72,74
Female,Group C,Some College,Standard,Completed,69,90,88
Female,Group B,Master's Degree,Standard,None,90,95,93
Male,Group A,Associate's Degree,Free/reduced,None,47,57,44
Male,Group C,Some College,Standard,None,76,78,75
Female,Group B,Associate's Degree,Standard,None,71,83,78
Female,Group B,Some College,Standard,Completed,88,95,92
Male,Group B,Some College,Free/reduced,None,40,43,39
Male,Group D,High School,Free/reduced,Completed,64,64,67
Female,Group B,High School,Free/reduced,None,38,60,50


In [0]:
%sql
SELECT
  MIN(`math score`)    AS min_math,    MAX(`math score`)    AS max_math,
  MIN(`reading score`) AS min_read,    MAX(`reading score`) AS max_read,
  MIN(`writing score`) AS min_write,   MAX(`writing score`) AS max_write,
  AVG(`math score`)    AS avg_math,
  AVG(`reading score`) AS avg_read,
  AVG(`writing score`) AS avg_write
FROM student_performance_insights.default.students_performance;

min_math,max_math,min_read,max_read,min_write,max_write,avg_math,avg_read,avg_write
0,100,17,100,10,100,66.089,69.169,68.054


In [0]:
%sql
-- 1.1 Null counts
SELECT
  SUM(CASE WHEN gender IS NULL OR TRIM(gender) = '' THEN 1 ELSE 0 END) AS null_gender,
  SUM(CASE WHEN `race/ethnicity` IS NULL OR TRIM(`race/ethnicity`) = '' THEN 1 ELSE 0 END) AS null_race_eth,
  SUM(CASE WHEN `parental level of education` IS NULL OR TRIM(`parental level of education`) = '' THEN 1 ELSE 0 END) AS null_parent_edu,
  SUM(CASE WHEN lunch IS NULL OR TRIM(lunch) = '' THEN 1 ELSE 0 END) AS null_lunch,
  SUM(CASE WHEN `test preparation course` IS NULL OR TRIM(`test preparation course`) = '' THEN 1 ELSE 0 END) AS null_prep,
  SUM(CASE WHEN `math score`    IS NULL THEN 1 ELSE 0 END) AS null_math,
  SUM(CASE WHEN `reading score` IS NULL THEN 1 ELSE 0 END) AS null_read,
  SUM(CASE WHEN `writing score` IS NULL THEN 1 ELSE 0 END) AS null_write
FROM student_performance_insights.default.students_performance; 

null_gender,null_race_eth,null_parent_edu,null_lunch,null_prep,null_math,null_read,null_write
0,0,0,0,0,0,0,0


In [0]:
%sql
-- 1.2 Distincts for categoricals
SELECT
  COUNT(DISTINCT gender)                        AS gender_distinct,
  COUNT(DISTINCT `race/ethnicity`)              AS race_eth_distinct,
  COUNT(DISTINCT `parental level of education`) AS parent_edu_distinct,
  COUNT(DISTINCT lunch)                         AS lunch_distinct,
  COUNT(DISTINCT `test preparation course`)     AS prep_distinct
FROM student_performance_insights.default.students_performance; 

gender_distinct,race_eth_distinct,parent_edu_distinct,lunch_distinct,prep_distinct
2,5,6,2,2
